In [ ]:
import pandas as pd, ipywidgets as widgets, numpy as np, matplotlib.pyplot as plt, io
from ipywidgets import interact, interactive, fixed, interact_manual

#Function that ensures a given antibody platemap is formatted correctly (by columns, not rows)
#Mutable function that changes the data frame passed in by argument "ab_sheet"
#ie A1->B1-->H1->A2-->B2-->A12-->H12
#First, separate the Well ID into separate "Plate Number","Format Row" and "Format Column" values 
#(Row is Letter, Column is Number)
#Sort the platemap into proper Plate -> Column -> Row format
#Returns nothing, mutates the ab_sheet antibody plate map
def format_platemap(ab_sheet):
    ab_sheet["Format Row"] = ab_sheet["Well ID"].str.slice(start = 0, stop = 1)
    numeric_col = ab_sheet["Well ID"].str.slice(start = 1)
    numeric_col = pd.to_numeric(numeric_col)
    ab_sheet["Format Column"] = numeric_col
    ab_sheet.sort_values(by=["Plate Number","Format Column", "Format Row"], inplace=True, ignore_index = True)
    return

#Average function that takes a series with n=1, n=2 replicates in sequential order within the series, and 
#Separates n=1 and n=2 replicates resets indeces, and calculates average
#Returns a series with the averaged values, removing the replicate wells
def average(series_to_average):
    firsts = series_to_average.iloc[::2]
    seconds = series_to_average.iloc[1::2]
    firsts.reset_index(inplace = True, drop = True)
    seconds.reset_index(inplace = True, drop = True)
    averaged_series = firsts + seconds
    averaged_series = averaged_series/2
    return averaged_series

#Takes a Pandas dataframe as an input. Takes every column from the dataframe and concatenates them into one single series
#Returns the new series that has all of the data in one column, starting from 1st column -> max # of columns
def columnize(df_to_columnize):
    return_column = pd.Series()
    for column in df_to_columnize.columns:
        colToAdd = df_to_columnize[column]
        return_column = pd.concat([return_column, colToAdd], axis = 0, ignore_index = True)
    return return_column

def run_analysis(b):
    elnname = list(eln_uploader.value.keys())[0]
    eln = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']))
    ab_sheet = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']), sheet_name = "Antibody Plate Map")
    functional_assay_type = assay_type_widget.value
    agonist_data = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']), 
                                 sheet_name = functional_assay_type + " Agonist")
    antagonist_data = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']), 
                                sheet_name = functional_assay_type + " Antagonist")

    #Create new DF for concatenating the Agonist, Antagonist signals, and the Antibody Plate Map
    format_platemap(ab_sheet)
    agonist_column, antagonist_column = columnize(agonist_data), columnize(antagonist_data)
    averageAgonists, averageAntagonists = average(agonist_column), average(antagonist_column)
    OutputSheet = pd.concat([averageAgonists, averageAntagonists, ab_sheet], axis = 1)
    OutputSheet.index += 1
    OutputSheet.rename(inplace = True, columns = {0: "Agonist Mode", 1: "Antagonist Mode"})
    OutputSheetCleaned = OutputSheet[["Agonist Mode", "Antagonist Mode", "DNA", "Concentration (mg/mL)"]]
    OutputSheetCleaned.to_excel(functional_assay_type + " Functional Assay Agonist and Antagonist Mode Analysis.xlsx", sheet_name = 'Analyzed Data')
    AgonistSort = OutputSheetCleaned.sort_values(by="Agonist Mode", ascending = False)
    display(AgonistSort)
    #AntagonistSort = OutputSheetCleaned.sort_values(by="Antagonist Mode", ascending = False)
    #OutputSheet


In [ ]:
eln_uploader = widgets.FileUpload(multiple = False)
assay_type_widget = widgets.Text(placeholder='Type something', description='Assay Type:', disabled=False)
run_button_widget = widgets.Button(description="Run Analysis")

display("Upload ELN w/ Functional Data and Antibodies Sheet (xlsx), and specify assay type")
display(eln_uploader)
display(assay_type_widget)
display(run_button_widget)
run_button_widget.on_click(run_analysis)

In [ ]:
# TopLeads = OutputSheetCleaned.nlargest(10, "Agonist Mode")
# fig, ax = plt.subplots()
# ax.scatter(x = OutputSheetCleaned.index, y=OutputSheetCleaned["Agonist Mode"], alpha = .2, color = "blue")
# ax.scatter(x = TopLeads.index, y = TopLeads["Agonist Mode"], alpha = .8, color = "blue", label = "Top Hits")
# ax.set_title(functional_assay_type + " Agonist Primary Screen")
# ax.set_xlabel("Antibody Number")
# ax.set_ylabel(functional_assay_type + " Readout")
# ax.legend()
